# 목차
* 데이터 전처리
* 딥러닝 모델 설계 및 훈련
* 딥러닝 평가하기
* 회고
* Reference

------------------
루브릭

아래의 기준을 바탕으로 프로젝트를 평가합니다.

* 평가문항	상세기준
1. 데이터의 전처리 및 구성과정이 체계적으로 진행되었는가?

  > 특수문자 제거, 토크나이저 생성, 패딩 처리의 작업들이 빠짐없이 진행되었는가?

2. 가사 텍스트 생성 모델이 정상적으로 동작하는가?

  > 텍스트 제너레이션 결과로 생성된 문장이 해석 가능한 문장인가?

3. 텍스트 생성모델이 안정적으로 학습되었는가?

  > 텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?

In [ ]:
import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import tensorflow

print(tensorflow.__version__)

2.8.2


# 데이터 전처리

## 데이터 불러오기

In [ ]:
import glob

txt_file_path = '/content/drive/MyDrive/아이펠 데이터/exp6 NLP/lyrics/*' 
txt_list = glob.glob(txt_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당

raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Let's stay together I, I'm I'm so in love with you", 'Whatever you want to do', 'Is all right with me']


## 데이터 정제

* *지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있겠죠.
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 를 권합니다.*

In [ ]:
# re: Regular Expression의 줄임말로, 파이썬 정규표현식을 사용하기 위한 모듈

import re 
import numpy as np

불러온 문장들을 딥러닝에 적용하기에 좋게 정제하는 과정을 가진다. 우선 대문자를 소문자로 바꾸고 일부 특수문자를 제거한다. 그리고 문장의 시작과 끝을 알리는 토큰을 추가한다.

이렇게 정제된 데이터에는 매우 많은 문장이 있으나 그 중 일부는 우리가 사용하기에 적합하지 않다. 예를 들어 빈 공백만이 존재하거나 극에서 ' : ' 만으로 이루어진 문장들이 그렇다. 따라서 이들을 제외한다. 또한 이번에는 가사를 만들고자 하므로 토큰의 개수가 15개 이상인 문장을 제외한다. 그런 긴 문장은 우리의 목적에 부합한 결과물을 만드는데 오히려 방해될 가능성이 높기 때문이다.

In [ ]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [ ]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) > 15: continue 
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> is what i need <end>',
 '<start> come on <end>',
 '<start> but baby , <end>',
 '<start> i love you so , <end>',
 '<start> even though , <end>',
 '<start> let me tell you <end>',
 '<start> oh the power <end>',
 '<start> power of love <end>',
 '<start> ha yeah <end>',
 '<start> say love <end>']

## 평가 데이터셋 분리

In [ ]:
import tensorflow as tf

14000개의 단어가 들어갈 수 있는 tokenizer를 만든다. 앞서 이미 문장들을 정제했고, 만약 여기에 포함되지 않는 단어가 온다면 그것은 <unk> 로 바꾼다. 이 때 입력에 들어가는 문장들의 토큰 수는 모두 같아야한다. 따라서 이에 미치지 못하는 짧은 문장들은 패딩을 붙여 모두 같은 길이를 가지도록 한다. padding='pre' 이면 문장 앞에 패딩이 붙고, padding=<post> 이면 문장 뒤에 패딩이 붙는다.

In [ ]:
def tokenize(corpus):
    # 14000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 14000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=14000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    # tokenizer.fit_on_texts(texts): 문자 데이터를 입력받아 리스트의 형태로 변환하는 메서드
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    # tokenizer.texts_to_sequences(texts): 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  34  42 ...   0   0   0]
 [  2  36  21 ...   0   0   0]
 [  2  59  24 ...   0   0   0]
 ...
 [  2 210   3 ...   0   0   0]
 [  2 210   3 ...   0   0   0]
 [  2 210   3 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fddb89f2a10>


In [ ]:
print(tensor[:3, :10])

[[ 2 34 42  4 87  3  0  0  0  0]
 [ 2 36 21  3  0  0  0  0  0  0]
 [ 2 59 24  5  3  0  0  0  0  0]]


In [ ]:
np.shape(tensor)

(14120, 13)

In [ ]:
# tokenizer.index_word: 현재 계산된 단어의 인덱스와 인덱스에 해당하는 단어를 dictionary 형대로 반환 (Ex. {index: '~~', index: '~~', ...})
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : .
7 : you
8 : oh
9 : it
10 : me


여기서 모든 문장이 인덱스 2로 시작하는 이유가 \<start> 때문이다.  
문장의 끝에 위치한 0은 사전에는 따로 나타나지 않지만 패딩 문자 \<pad> 일 것이다.  

In [ ]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[ 2 34 42  4 87  3  0  0  0  0  0  0]
[34 42  4 87  3  0  0  0  0  0  0  0]


In [ ]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=42)

In [ ]:
print('enc_train:', enc_train.shape)
print('dec_train:', dec_train.shape)

enc_train: (11296, 12)
dec_train: (11296, 12)


In [ ]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer.num_words: 주어진 데이터의 문장들에서 빈도수가 높은 n개의 단어만 선택
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 12), dtype=tf.int32, name=None), TensorSpec(shape=(256, 12), dtype=tf.int32, name=None))>

# 딥러닝 모델 설계 및 훈련

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy( 
    from_logits=True, reduction='none') # 클래스 분류 문제에서 softmax 함수를 거치면 from_logits = False(default값),그렇지 않으면 from_logits = True.

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다. 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!   
embedding_size = 256 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

In [ ]:
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
lyricist(src_sample)

<tf.Tensor: shape=(256, 12, 14001), dtype=float32, numpy=
array([[[ 2.77788029e-04, -1.27151594e-04, -1.74182715e-04, ...,
         -1.47247629e-04,  7.12201290e-05,  1.03006561e-04],
        [ 3.08117509e-04,  6.41052102e-05, -2.66175572e-04, ...,
         -1.14040879e-04,  2.15024469e-04,  1.75263587e-04],
        [ 2.44525087e-04,  2.20720700e-04, -5.60876251e-05, ...,
          7.37074733e-05,  3.34380195e-04,  2.79293803e-04],
        ...,
        [ 1.24193588e-03,  1.19125170e-05, -2.49085715e-04, ...,
          7.36151182e-04, -2.76153628e-03, -6.38602942e-04],
        [ 1.23954308e-03,  9.17265152e-06, -2.36481428e-04, ...,
          9.19198093e-04, -3.17613897e-03, -7.18313735e-04],
        [ 1.20552292e-03,  1.33618287e-05, -2.04468844e-04, ...,
          1.11503259e-03, -3.54487752e-03, -7.79030670e-04]],

       [[ 2.77788029e-04, -1.27151594e-04, -1.74182715e-04, ...,
         -1.47247629e-04,  7.12201290e-05,  1.03006561e-04],
        [ 4.73459688e-04, -3.33042153e-05, -7

In [ ]:
lyricist.summary()

Model: "text_generator_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     multiple                  3584256   
                                                                 
 lstm_10 (LSTM)              multiple                  5246976   
                                                                 
 lstm_11 (LSTM)              multiple                  8392704   
                                                                 
 dense_5 (Dense)             multiple                  14351025  
                                                                 
Total params: 31,574,961
Trainable params: 31,574,961
Non-trainable params: 0
_________________________________________________________________


* 아래 셀 실행 전 반드시 런타임 유형 변경할 것

In [ ]:
# Adam

optimizer = tf.keras.optimizers.Adam() # Adam은 현재 가장 많이 사용하는 옵티마이저이다. 자세한 내용은 차차 배운다.
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다. 
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
lyricist.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
lyricist.fit(dataset, epochs=30, verbose=1) # 만들어둔 데이터셋으로 모델을 학습한다. 30번 학습을 반복하겠다는 의미다.

# verbose = 1 이면 학습 과정을 간략하게 보여준다.

Epoch 1/30
55/55 [==============================] - 7s 85ms/step - loss: 2.8308
Epoch 2/30
55/55 [==============================] - 5s 85ms/step - loss: 1.6788
Epoch 3/30
55/55 [==============================] - 5s 86ms/step - loss: 1.4633
Epoch 4/30
55/55 [==============================] - 5s 85ms/step - loss: 1.4000
Epoch 5/30
55/55 [==============================] - 5s 85ms/step - loss: 1.3705
Epoch 6/30
55/55 [==============================] - 5s 86ms/step - loss: 1.3474
Epoch 7/30
55/55 [==============================] - 5s 87ms/step - loss: 1.3212
Epoch 8/30
55/55 [==============================] - 5s 85ms/step - loss: 1.2873
Epoch 9/30
55/55 [==============================] - 5s 86ms/step - loss: 1.2637
Epoch 10/30
55/55 [==============================] - 5s 86ms/step - loss: 1.3524
Epoch 11/30
55/55 [==============================] - 5s 86ms/step - loss: 1.2311
Epoch 12/30
55/55 [==============================] - 5s 86ms/step - loss: 1.2069
Epoch 13/30
55/55 [==================

# 딥러닝 평가하기

## Adam

In [ ]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [ ]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 ilove 로 시작되는 문장을 생성

'<start> i love you <end> '

In [ ]:
generate_text(lyricist, tokenizer, init_sentence="<start> i wanna", max_len=20)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 ilove 로 시작되는 문장을 생성

'<start> i wanna mingle <end> '

## RMSprop

In [ ]:
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
# 다른 옵티마이저 사용
# RMSprop

optimizer = tf.keras.optimizers.RMSprop() 
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다. 
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
lyricist.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
lyricist.fit(dataset, epochs=30, verbose=1) # 만들어둔 데이터셋으로 모델을 학습한다. 30번 학습을 반복하겠다는 의미다.

# verbose = 1 이면 학습 과정을 간략하게 보여준다.

Epoch 1/30
55/55 [==============================] - 9s 88ms/step - loss: 2.2102
Epoch 2/30
55/55 [==============================] - 5s 88ms/step - loss: 1.4654
Epoch 3/30
55/55 [==============================] - 5s 88ms/step - loss: 1.3538
Epoch 4/30
55/55 [==============================] - 5s 88ms/step - loss: 1.2893
Epoch 5/30
55/55 [==============================] - 5s 88ms/step - loss: 1.2302
Epoch 6/30
55/55 [==============================] - 5s 88ms/step - loss: 1.1798
Epoch 7/30
55/55 [==============================] - 5s 88ms/step - loss: 1.1340
Epoch 8/30
55/55 [==============================] - 5s 88ms/step - loss: 1.0958
Epoch 9/30
55/55 [==============================] - 5s 88ms/step - loss: 1.0607
Epoch 10/30
55/55 [==============================] - 5s 88ms/step - loss: 1.0279
Epoch 11/30
55/55 [==============================] - 5s 88ms/step - loss: 0.9998
Epoch 12/30
55/55 [==============================] - 5s 88ms/step - loss: 0.9735
Epoch 13/30
55/55 [==================

In [ ]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 ilove 로 시작되는 문장을 생성

'<start> i love you <end> '

In [ ]:
generate_text(lyricist, tokenizer, init_sentence="<start> i wanna", max_len=20)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 ilove 로 시작되는 문장을 생성

'<start> i wanna do it <end> '

## Adgrad

In [ ]:
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
# 다른 옵티마이저 사용
# Adagrad

optimizer = tf.keras.optimizers.Adagrad() 
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다. 
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
lyricist.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
lyricist.fit(dataset, epochs=60, verbose=1) # 만들어둔 데이터셋으로 모델을 학습한다. 60번 학습을 반복하겠다는 의미다.

# verbose = 1 이면 학습 과정을 간략하게 보여준다.

Epoch 1/60
55/55 [==============================] - 7s 85ms/step - loss: 1.4810
Epoch 2/60
55/55 [==============================] - 5s 85ms/step - loss: 1.3912
Epoch 3/60
55/55 [==============================] - 5s 85ms/step - loss: 1.3826
Epoch 4/60
55/55 [==============================] - 5s 85ms/step - loss: 1.3768
Epoch 5/60
55/55 [==============================] - 5s 85ms/step - loss: 1.3713
Epoch 6/60
55/55 [==============================] - 5s 85ms/step - loss: 1.3675
Epoch 7/60
55/55 [==============================] - 5s 85ms/step - loss: 1.3636
Epoch 8/60
55/55 [==============================] - 5s 85ms/step - loss: 1.3599
Epoch 9/60
55/55 [==============================] - 5s 85ms/step - loss: 1.3570
Epoch 10/60
55/55 [==============================] - 5s 85ms/step - loss: 1.3542
Epoch 11/60
55/55 [==============================] - 5s 85ms/step - loss: 1.3514
Epoch 12/60
55/55 [==============================] - 5s 85ms/step - loss: 1.3494
Epoch 13/60
55/55 [==================

In [ ]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 ilove 로 시작되는 문장을 생성

'<start> i love <end> '

In [ ]:
generate_text(lyricist, tokenizer, init_sentence="<start> i wanna", max_len=20)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 ilove 로 시작되는 문장을 생성

'<start> i wanna you <end> '

In [ ]:
# Adagrad
# 60회 학습 추가 반복

optimizer = tf.keras.optimizers.Adagrad() 
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다. 
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
lyricist.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
lyricist.fit(dataset, epochs=60, verbose=1) # 만들어둔 데이터셋으로 모델을 학습한다. 60번 학습을 반복하겠다는 의미다.

# verbose = 1 이면 학습 과정을 간략하게 보여준다.

Epoch 1/60
55/55 [==============================] - 8s 85ms/step - loss: 1.4047
Epoch 2/60
55/55 [==============================] - 5s 85ms/step - loss: 1.2870
Epoch 3/60
55/55 [==============================] - 5s 85ms/step - loss: 1.2815
Epoch 4/60
55/55 [==============================] - 5s 85ms/step - loss: 1.2783
Epoch 5/60
55/55 [==============================] - 5s 85ms/step - loss: 1.2753
Epoch 6/60
55/55 [==============================] - 5s 85ms/step - loss: 1.2729
Epoch 7/60
55/55 [==============================] - 5s 85ms/step - loss: 1.2702
Epoch 8/60
55/55 [==============================] - 5s 85ms/step - loss: 1.2685
Epoch 9/60
55/55 [==============================] - 5s 85ms/step - loss: 1.2661
Epoch 10/60
55/55 [==============================] - 5s 85ms/step - loss: 1.2643
Epoch 11/60
55/55 [==============================] - 5s 85ms/step - loss: 1.2630
Epoch 12/60
55/55 [==============================] - 5s 85ms/step - loss: 1.2610
Epoch 13/60
55/55 [==================

In [ ]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 ilove 로 시작되는 문장을 생성

'<start> i love you <end> '

In [ ]:
generate_text(lyricist, tokenizer, init_sentence="<start> i wanna", max_len=20)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 ilove 로 시작되는 문장을 생성

'<start> i wanna a da <end> '

# 회고

오늘은 자연어 처리를 이용해 주어진 단어들을 받으면 그에 이어서 자동으로 다음 단어를 추가하는 일을 했다. 자연어 처리는 우리가 일상적으로 사용하는 언어를 컴퓨터가 해석할 수 있도록 해야하므로 그것을 위한 데이터를 준비하는 것이 가장 중요하다.

우선 내가 가진 데이터들을 전처리하는 작업이 필요하다. 이 때 불필요한 요소들과 문장들을 제거한다. 자연어의 특성상 수많은 데이터가 포함되어 있는데 이 때 불순물들을 걸러내지 못 한다면 딥러닝 모델의 목적을 달성하는 데에 방해가 될 수 있기 때문이다.

그러고나면 기존에 존재하는 단어 사전을 가지고 각각의 단어들을 인덱스 토큰으로 바꿔준다. 단어 그 자체를 컴퓨터가 인식하는 것은 비효율적이므로 토큰으로 바꿔주고 레이어에 들어갈 수 있도록 모든 문장을 같은 수의 단어를 가지도록 한다. 짧은 문장에는 패딩 문자를 채워서 수를 통일하는 것이다. 이렇게 변환된 문장들을 embedding 레이어에 넣게 되면 각 단어들은 벡터화되어 여타 딥러닝 모델들과 마찬가지로 매개변수들을 학습하게 된다.

여기서 중요한 것은 단어들을 벡터로 만든다는 것이다. 앞서 인덱스로 토큰화하긴 했으나 이 토큰들은 단어의 의미나, 단어 사이의 관계를 설명해주지 않는다. 따라서 단어의 의미를 여러 축을 가진 벡터들의 합으로 표현하는 것이다. 이렇게 한다면 그 뜻 뿐만 아니라 비슷한 단어들끼리 묶거나 유사한 단어들 간의 미묘한 차이를 포착하는 등 훨씬 더 정교한 작업을 할 수 있게 된다. 딥러닝은 바로 단어들의 벡터를 찾아내는 작업을 한다.

이 때 모델에 optimizer가 존재하는데 이는 매개변수를 찾아가는 방식을 정의한다. 분명 같은 딥러닝 모델을 사용하더라도, 이 optimizer가 달라진다면 학습속도나 효율 그리고 결과물까지 달라질 수 있다. 일반적으로 Adam을 사용하지만 이번에는 RMSprop도 비슷하거나 더 나은 결과물을 (더 낮은 loss) 제공했으며 Adgrad는 점점 학습속도가 느려지는 특성상 다른 optimizer의 2배의 epoch을 진행했음에도 제대로 된 결과물이 나오지 않았다. 여기에 60회를 추가로 진행했을 때도 여전히 i wanna a da 라는 문장을 출력하며 불완전한 모습을 보였다.

만약 나중에 자연어 처리 모델을 사용하게 된다면 필요와 상황에 맞게 optimizer를 사용해야만 빠른 속도와 좋은 결과물을 얻을 수 있을 것이다. 이 때 똑같이 학습이 잘 되었어도 결과물에는 작은 차이가 존재하기 때문에 각 특성을 미리 살펴보는 것이 중요하다고 할 수 있겠다.



# Reference

https://blog.naver.com/gypsi12/222657990749 Tokenizer

https://www.tensorflow.org/api_docs/python/tf/data/Dataset Dataset

https://hwiyong.tistory.com/335 from_logits

https://zzcojoa.tistory.com/108 Optimizers

https://velog.io/@yookyungkho/%EB%94%A5%EB%9F%AC%EB%8B%9D-%EC%98%B5%ED%8B%B0%EB%A7%88%EC%9D%B4%EC%A0%80-%EC%A0%95%EB%B3%B5%EA%B8%B0%EB%B6%80%EC%A0%9C-CS231n-Lecture7-Review Otimizers 2